Robot Localization with Python and Particle Filters
===================================================

**Import libraries and load map**

In [1]:
import numpy as np
import cv2
map = cv2.imread("map.png", 0) # 0 for greyscale
HEIGHT, WIDTH = map.shape
print(map)

rx,ry,rtheta = (WIDTH/4, HEIGHT/4, 0)

[[ 79  80  82 ... 133 148 156]
 [ 78  79  81 ... 138 156 164]
 [ 76  77  79 ... 147 170 180]
 ...
 [181 181 182 ... 174 172 171]
 [178 178 178 ... 180 179 178]
 [177 177 177 ... 183 182 182]]


**Map coordinate system**

![title](coords.png)

CAUTION: The terrain height at X,Y coordinates is map(Y,X).

**Read keyboard input**

In [2]:
STEP = 5
TURN = np.radians(25)
def get_input():
    fwd = 0
    turn = 0
    halt = False
    k = cv2.waitKey(0)
    if k == 82:        # UP arrow 
        fwd = STEP
    elif k == 83:      # Right arrow
        turn = TURN
    elif k == 81:      # Left arrow
        turn = -TURN   # Opposite direction
    else:
        halt = TRUE
    return fwd, turn, halt

**Move the robot, with Gausssian noise**

![title](gaussian.png)

In the real world, the terrain is uneven and the robot control is not perfect. So, there is some uncertainity about how the robot will move according to our command. We can simulate this by adding a Gaussian noise to the stepping and steering values. Above, the green line at the center is greek letter 'mu' which represents the forward value that we want to command our robot. But the distance that it actually moves maybe a bit more, or a bit less.

The blue curve here represents the probability of the values we can get and sigma here is the standard deviation. This controls how widely spread out the noise will be.

In [3]:
# Updating robot position according to the keyboard command
SIGMA_STEP = 0.5           # forward movement
SIGMA_TURN = np.radians(5) # steering command 

def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = np.random.normal(fwd, SIGMA_STEP, 1) #1 value to get back
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)
    print("fwd_noisy = ", fwd_noisy)
    
    turn_noisy = np.random.normal(turn, SIGMA_TURN, 1)
    rtheta += turn_noisy
    print("turn_noisy = ", np.degrees(turn_noisy))
    
    
    return rx, ry, rtheta
# RUN ALL CELLS

**Initialize particle cloud**

In [4]:
NUM_PARTICLES = 3000
def init():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles *= np.array((WIDTH, HEIGHT, np.radians(360)))
    return particles

**Move the particles**

In [5]:
def move_particles(particles, fwd, turn):
    particles[:,0] += fwd * np.cos(particles[:,2]) # X position, 1st column
    particles[:,1] += fwd * np.sin(particles[:,2]) # Y position, 2nd column
    particles[:,2] += turn
    
    particles[:,0] = np.clip(particles[:,0], 0.0, WIDTH-1)
    particles[:,1] = np.clip(particles[:,1], 0.0, HEIGHT-1)
    return particles

**Get value from robot's sensor**

In [6]:
SIGMA_SENSOR = 2
def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)
    if noisy:
        return np.random.normal(map[y,x], SIGMA_SENSOR, 1)
    return map[y,x]

**Compute particle weights**

In [7]:
def compute_weights(particles, robot_sensor):    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        elevation = sense(particles[i,0], particles[i,1], noisy = FALSE)
        errors[i] = abs(robot_sensor - elevation)
    weights = np.max(errors) - errors
    
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH - 1) |
        (particles[:,0] == 0) |
        (particles[:,0] == HEIGHT -1) 
    ] = 0.0
    weights = weights ** 3
    return weights

**Resample the particles**

In [8]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights)
    new_index = np.random.choice(
        NUM_PARTICLES,
        size = NUM_PARTICLES,
        p = probabilities  
    )
    particles = particles[new_index,:]
    return particles

**Add noise to the particles**

In [9]:
SIGMA_POS = 2
SIGMA_TURN = np.radians(10)

def add_noise(particles):
    noise = np.concatenate(
        (
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_TURN, (NUM_PARTICLES, 1)),
        ),
        axis=1
    )
    particles += noise
    return particles

**Display robot, particles and best guess**

In [10]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    # Display particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, 
                       (int(particles[i,0]), int(particles[i,1])), 
                       1, 
                       (255,0,0), 
                       1)
        
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    # Display best guess
    if len(particles) > 0:
        px = np.mean(particles[:,0])
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

**Main routine**

In [ ]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)        
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    
cv2.destroyAllWindows()                        
